The purpose of this is to find out what we should test to make sure that the extending works correctly (i.e. groups are only registered if not inherited).

- Test registering a copy/extension of a type that includes group (already changed in new branch)
- Test on `is_inherited_*` methods

In [1]:
import pynwb

/Users/ageorgou/anaconda/envs/pynwb-local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ns_cat = pynwb.__NS_CATALOG

In [3]:
dev_spec = ns_cat.get_spec(pynwb.CORE_NAMESPACE, 'Device')

In [4]:
dev_spec

{'attributes': [{'doc': "Value is 'A recording device e.g. amplifier'.",
   'dtype': 'text',
   'name': 'help',
   'value': 'A recording device e.g. amplifier'}],
 'doc': 'One of possibly many. Information about device and device description.',
 'neurodata_type_def': 'Device',
 'neurodata_type_inc': 'NWBContainer',
 'quantity': '+'}

In [5]:
dev_spec.get_attribute('help')

{'doc': "Value is 'A recording device e.g. amplifier'.",
 'dtype': 'text',
 'name': 'help',
 'value': 'A recording device e.g. amplifier'}

In [6]:
file_spec = ns_cat.get_spec(pynwb.CORE_NAMESPACE, 'NWBFile')

In [7]:
file_spec.is_inherited_type('devices')

True

In [8]:
file_spec.parent

In [9]:
type(file_spec)

pynwb.spec.NWBGroupSpec

In [10]:
[g.name for g in file_spec.groups]

['acquisition',
 'analysis',
 'processing',
 'stimulus',
 'general',
 'intervals',
 'units']

In [11]:
gen_spec = file_spec.get_group('general')

In [12]:
[g.name for g in gen_spec.groups]

['devices',
 'specifications',
 'subject',
 'extracellular_ephys',
 'intracellular_ephys',
 'optogenetics',
 'optophysiology']

In [13]:
dev_group_spec = gen_spec.get_group('devices')

In [14]:
dev_group_spec.groups[0]

{'attributes': [{'doc': "Value is 'A recording device e.g. amplifier'.",
   'dtype': 'text',
   'name': 'help',
   'value': 'A recording device e.g. amplifier'}],
 'doc': 'One of possibly many. Information about device and device description.',
 'neurodata_type_def': 'Device',
 'neurodata_type_inc': 'NWBContainer',
 'quantity': '+'}

In [15]:
dev_group_spec.groups[0].attributes

({'doc': "Value is 'A recording device e.g. amplifier'.",
  'dtype': 'text',
  'name': 'help',
  'value': 'A recording device e.g. amplifier'},)

In [19]:
type(dev_group_spec.groups[0])

pynwb.spec.NWBGroupSpec

In [16]:
dev_group_spec.groups[0]['neurodata_type_inc']

'NWBContainer'

## Test 1

We know that this has to pass (it breaks with the original `pynwb`):

In [17]:
from pynwb import load_namespaces
from pynwb.spec import NWBGroupSpec, NWBNamespaceBuilder

ns_builder = NWBNamespaceBuilder("A pointless extension", "foospace")

foofile_spec = NWBGroupSpec(
    "An NWBFile but with a different name",
    neurodata_type_inc="NWBFile",
    neurodata_type_def="FooFile"
)

# Write extension
file_source = 'foofile.yaml'
ns_path = 'foo.namespace.yaml'
ns_builder.add_spec(file_source, foofile_spec)
ns_builder.export(ns_path)

# Error raised here
load_namespaces(ns_path)

ValueError: 'Device' - cannot overwrite existing specification

Actually, this is very similar to an [already existing file](./tests/build_fake_extension.py). The latter, however, does not include the last part (`load_namespaces`), which is where the error occurs, and it doesn't seem to be invoked form anywhere.

## Test 2

There is a test I have modified the `pynwb` suite. It fails with the original code version, but works with my current version (`f32f446a`) which is still not correct (Test 1 above fails with it). It therefore needs to be reworked, but could be a good extension to the existing ones. The changes currently look like this:

```diff
--- a/tests/unit/form_tests/spec_tests/test_load_namespace.py
+++ b/tests/unit/form_tests/spec_tests/test_load_namespace.py
@@ -36,10 +36,17 @@ class TestSpecLoad(unittest.TestCase):
                         linkable=True,
                         data_type_def='VoltageArray')
         ]
+        self.groups = [
+            GroupSpec('my first (empty) group',  # noqa: F405
+                       name='trivial_group',
+                       linkable=False,
+                       data_type_def='EmptyGroup')
+        ]
         self.spec = GroupSpec('A test group',  # noqa: F405
                               name='root_constructor_nwbtype',
                               datasets=self.datasets,
                               attributes=self.attributes,
+                              groups=self.groups,
                               linkable=False,
                               data_type_def='EphysData')```

## Test 3

Something that checks the extra bit needed to make Test 2 fail for an incorrect version, so that we have an equivalent test to 1, but that can be incorporated in the existing `pynwb` test suite.

## More experimentation